In [1]:
import os, sys, json, random, shutil
import albumentations as A
from PIL import Image
import random
import numpy as np
import threading

lock = threading.Lock()

In [2]:
#print("bounding boxes loading")
#with open("../../../CameraTraps/snapshotserengeti-unzipped/SnapshotSerengetiBboxes_20190903.json") as f:
#    bboxes = json.load(f)

with open("../../../CameraTraps/snapshotserengeti-unzipped/SnapshotSerengeti_S1-6_v2.1_categories_only.json") as f:
    metadata = json.load(f)

In [3]:
transform = A.Compose([
    A.Rotate(limit=(-90, 90), p=0.7),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5)
])


def copyFile(src, dst, name, numAugs, testing, n):
    if os.path.exists(src):
        if not os.path.exists(dst):
            os.makedirs(dst)
        
        im = Image.open(src)
        im = im.crop((0, 0, 2048, 1420))
        im = im.resize((1024, 1024))
        if(testing):
            while True:
                    if lock.acquire(timeout=1):  # acquire the lock with a timeout of 1 second
                        try: 
                            im.save(dst + name + ".JPG")
                            break  # exit the loop after successfully acquiring the lock and saving the file
                        finally:
                            lock.release()  # always release the lock in a finally block
                    else:
                        print("Unable to acquire lock within 1 second. Trying again...")

        else:
            if(numAugs < 2):
                transformed = transform(image=np.array(im))['image']
                transformed = Image.fromarray(transformed)
                while True:
                    if lock.acquire(timeout=1):  # acquire the lock with a timeout of 1 second
                        try:
                            transformed.save(dst + name + ".JPG" )
                            break  # exit the loop after successfully acquiring the lock and saving the file
                        finally:
                            lock.release()  # always release the lock in a finally block
                    else:
                        print("Unable to acquire lock within 1 second. Trying again...")
            else:
                while True:
                    if lock.acquire(timeout=1):  # acquire the lock with a timeout of 1 second
                        try: 
                            im.save(dst + name + ".JPG")
                            break  # exit the loop after successfully acquiring the lock and saving the file
                        finally:
                            lock.release()  # always release the lock in a finally block
                    else:
                        print("Unable to acquire lock within 1 second. Trying again...")

            numAugs = int(min(n/50, numAugs))

            for i in range(1, numAugs):
                transformed = transform(image=np.array(im))['image']
                transformed = Image.fromarray(transformed)
                while True:
                    if lock.acquire(timeout=1):  # acquire the lock with a timeout of 1 second
                        try:
                            transformed.save(dst + name + f"_{i}" + ".JPG")
                            break  # exit the loop after successfully acquiring the lock and saving the file
                        finally:
                            lock.release()  # always release the lock in a finally block
                    else:
                        print("Unable to acquire lock within 1 second. Trying again...")

In [4]:
def process_season(season, thrd):
    bp = "../../../CameraTraps/snapshotserengeti-unzipped/"
    itr = 0
    d = {}
    total = 0
    for anot in metadata['annotations']:
        if(anot['season'] == season):
            if(metadata['categories'][anot['category_id']]['name'] not in d.keys()):
                d[metadata['categories'][anot['category_id']]['name']] = {'num': 1, 'loc': [anot['image_id']]}
            else:
                d[metadata['categories'][anot['category_id']]['name']]['num'] += 1
                d[metadata['categories'][anot['category_id']]['name']]['loc'].append(anot['image_id'])
            total += 1

    d.pop('empty')
    ns[thrd] = 1e5
    for key in d.keys():
        ns[thrd] = min(d[key]['num'], ns[thrd])
    ns[thrd] = 1000*ns[thrd]

    d1 = {}
    for key in d.keys():
        if(d[key]['num'] > ns[thrd]):
            l = random.sample(d[key]['loc'], ns[thrd])
        else:
            l = d[key]['loc']
        d1[key] = {'num': len(l), 'loc': l}

    total = 0
    for key in d1.keys():
        total += d1[key]['num']
    itr = 0

    for key in d1.keys():
        for i in range(d1[key]['num']):
            try:
                itr += 1
                r = (random.random())
                if(r >= .8):
                    copyFile(
                            bp + d1[key]['loc'][i] + ".JPG", 
                            "../data/SS/test/" + key + "/", 
                            d1[key]['loc'][i].split("/")[-1], 
                            int(round(1/(d1[key]['num']/ns[thrd]))),
                            True,
                            ns[thrd]
                            )
                else:
                    copyFile(
                            bp + d1[key]['loc'][i] + ".JPG", 
                            "../data/SS/train/" + key + "/", 
                            d1[key]['loc'][i].split("/")[-1], 
                            int(round(1/(d1[key]['num']/ns[thrd]))),
                            False,
                            ns[thrd]
                            )
            except OSError:
                print(d1[key]['loc'][i], i, itr, r)
            
            if(itr % (total//1000) == 0):
                print(round(itr/total*100, 1), "%,", season)

seasons = ['S2']#, 'S2', 'S3', 'S4', 'S5', 'S6']
ns = [0, 0, 0, 0, 0, 0]
threads = []

for season in seasons:
    t = threading.Thread(target=process_season, args=(season, len(threads)))
    t.start()
    threads.append(t)

# Wait for all threads to finish
for t in threads:
    t.join()

0.1 %, S2
0.2 %, S2
0.3 %, S2
0.4 %, S2
0.5 %, S2
0.6 %, S2
0.7 %, S2
0.8 %, S2
0.9 %, S2
1.0 %, S2
1.1 %, S2
1.2 %, S2
1.3 %, S2
1.4 %, S2
1.5 %, S2
1.6 %, S2
1.7 %, S2
1.8 %, S2
1.9 %, S2
2.0 %, S2
2.1 %, S2
2.2 %, S2
2.3 %, S2
2.4 %, S2
2.5 %, S2
2.6 %, S2
2.7 %, S2
2.8 %, S2
2.9 %, S2
3.0 %, S2
3.1 %, S2
3.2 %, S2
3.3 %, S2
3.4 %, S2
3.5 %, S2
3.6 %, S2
3.7 %, S2
3.8 %, S2
3.9 %, S2
4.0 %, S2
4.1 %, S2
4.2 %, S2
4.3 %, S2
4.4 %, S2
4.5 %, S2
4.6 %, S2
4.7 %, S2
4.8 %, S2
4.9 %, S2
5.0 %, S2
5.1 %, S2
5.2 %, S2
5.3 %, S2
5.4 %, S2
5.5 %, S2
5.6 %, S2
5.7 %, S2
5.8 %, S2
5.9 %, S2
6.0 %, S2
6.1 %, S2
6.2 %, S2
6.3 %, S2
6.4 %, S2
6.5 %, S2
6.6 %, S2
6.7 %, S2
6.8 %, S2
6.9 %, S2
7.0 %, S2
7.1 %, S2
7.2 %, S2
7.2 %, S2
7.3 %, S2
7.4 %, S2
7.5 %, S2
7.6 %, S2
7.7 %, S2
7.8 %, S2
7.9 %, S2
8.0 %, S2
8.1 %, S2
8.2 %, S2
8.3 %, S2
8.4 %, S2
8.5 %, S2
8.6 %, S2
8.7 %, S2
8.8 %, S2
8.9 %, S2
9.0 %, S2
9.1 %, S2
9.2 %, S2
9.3 %, S2
9.4 %, S2
9.5 %, S2
9.6 %, S2
9.7 %, S2
9.8 %, S2
9.9 %, S2
